# Sign Language Recognition Demo

This jupyter notebook is a quick Sign Language Recognition Demo. The test data is saved under `data/` and is listed under `lists/`. The trained HMM is stored under `models/`. The videos are being displayed from `videos/`.

In [1]:
%load_ext autoreload
%autoreload 2

## Feature Extraction Pipeline

This part of the notebook covers feature extraction from raw videos. We assume that our raw videos are sourced from the ASL Capture App. This app records whole sessions of the user signing words. Each session consists of 20 signs for 20 different words. The raw video must therefore be split up and then have mediapipe features extracted before we can train the models. The mediapipe features are then converted into HTK features.

### Slicing the Videos

To slice the videos, we first add our raw videos to `raw/`. Once this is done, run the `decode.py` script below. You can retrieve a list of signs that whose videos will be extracted without actually extracting them by passing the `only_print_signs` flag. We run it with capture magic, since there is a lot of output. To view the output, remove the magic command.

In [2]:
%%capture

source_dir = 'raw/'
dest_dir = 'split/'
num_threads = 32

%run decode.py --backup_dir $source_dir --dest_dir $dest_dir --num_threads $num_threads

We can count the sliced up videos.

In [3]:
sliced_video_files = !ls split/
print(len(sliced_video_files) - 1) # Leave out the error directory

100


We can also list the sliced up videos. Note the naming convention. The elements in the name are delimited by a hyphen (`-`). The first element is the user ID set in the Capture App. The second element is the word being signed. The third element is the dat and time at which the session was recorded (`YYYY_MM_DD_HH_MM_SS`). The fourth and last element, before the file extension, is the session ID followed by the number of times the sign was recorded in that session (`0` indexed). Ignore the error directory listed.

In [4]:
!ls split/

00001-puzzle-2022_09_19_15_42_00.307-0.mp4
00001-puzzle-2022_09_19_15_42_00.307-1.mp4
00001-puzzle-2022_09_19_15_42_00.307-10.mp4
00001-puzzle-2022_09_19_15_42_00.307-11.mp4
00001-puzzle-2022_09_19_15_42_00.307-12.mp4
00001-puzzle-2022_09_19_15_42_00.307-13.mp4
00001-puzzle-2022_09_19_15_42_00.307-14.mp4
00001-puzzle-2022_09_19_15_42_00.307-15.mp4
00001-puzzle-2022_09_19_15_42_00.307-16.mp4
00001-puzzle-2022_09_19_15_42_00.307-17.mp4
00001-puzzle-2022_09_19_15_42_00.307-18.mp4
00001-puzzle-2022_09_19_15_42_00.307-19.mp4
00001-puzzle-2022_09_19_15_42_00.307-2.mp4
00001-puzzle-2022_09_19_15_42_00.307-3.mp4
00001-puzzle-2022_09_19_15_42_00.307-4.mp4
00001-puzzle-2022_09_19_15_42_00.307-5.mp4
00001-puzzle-2022_09_19_15_42_00.307-6.mp4
00001-puzzle-2022_09_19_15_42_00.307-7.mp4
00001-puzzle-2022_09_19_15_42_00.307-8.mp4
00001-puzzle-2022_09_19_15_42_00.307-9.mp4
00001-sleep-2022_09_19_15_42_00.307-0.mp4
00001-sleep-2022_09_19_15_42_00.307-1.mp4
00001-sleep-2022_09_19_15_42_00.307-10.mp4
000

Finally, we can play any videos listed above. To do so, define the `video_file` variable below as the name of the video file.

In [5]:
import os
from IPython.display import Video

video_dir = 'split'
video_file = '00001-sleep-2022_09_19_15_42_00.307-18.mp4'

video_path = os.path.join(video_dir, video_file)
Video(video_path, height=540, width=360)

In the next step, we extract mediapipe features from the split videos and store them in `mediapipe/`. To do this, we call `mediapipe_convert.py`. This script generates mediapipe features for the split videos and stores them in the output directory. The directory hierarchy under the output directory is organized like so: The first level contains a directory for each user named: `[Capture App User ID]-singlesign/`. Beneath this level, the script creates a directory for each signed word named: `[sign]/`. Finally, at the last level the script creates a directory named after the date and time of each session in which the sign was recorded. The naming generally looks like this: `[YYYY_MM_DD_HH_MM_SS]` This final directory contains a data file with the mediapipe features.

In [6]:
%%capture

source_dir = 'split/'
dest_dir = 'mediapipe/'

%run mediapipe_convert.py --inputDirectory $source_dir --outputDirectory $dest_dir --noMark

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


We can count the number of signs for which we have mediapipe features.

In [7]:
!ls mediapipe/00001-singlesign/ | wc -l

5


For the final data preparation step, we convert our mediapipe features into a format usable by the HTK toolkit.

In [8]:
from utils.utils import load_json
from prepare_data import prepare_data

# This file should contain a path near the end thatpoints to the mediapipe features above.
features_file = 'configs/features.json'

# Load features
raw_data = load_json(features_file)
    
prepare_data(raw_data, isSingleWord=True)

/root/Mobile-Data-Processing-Pipeline/demo/mediapipe
Generating ark/htk using select_features data model


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:29<00:00,  3.35it/s]


Features Num Rows Distribution:  defaultdict(<class 'int'>, {53: 5, 50: 5, 48: 8, 57: 2, 41: 3, 45: 6, 49: 2, 47: 4, 76: 1, 46: 2, 58: 8, 100: 1, 44: 6, 42: 3, 37: 4, 43: 7, 51: 3, 52: 1, 31: 1, 54: 2, 62: 4, 73: 1, 63: 1, 71: 1, 61: 3, 70: 1, 55: 2, 60: 2, 36: 1, 38: 1, 39: 1, 40: 1, 28: 1, 66: 1, 56: 2, 64: 1, 65: 2})
.ark files created
Creating .htk files


  0%|                                                                                                                                                                                                                                                         | 0/100 [00:00<?, ?it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.puzzle.singlesign.00000008.ark 
LOG (copy-feats-to-htk[5.5.1056~1-f6f4c]:main():copy-feats-to-htk.cc:134) 1 HTK feature files generated in the direcory: data/htk
  1%|██▍                                                                                                                                                                                                                                              | 1/100 [00:00<01:27,  1.13it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.puzzle.singlesign.00000015.ark 
LOG (copy-feats-

 15%|████████████████████████████████████                                                                                                                                                                                                            | 15/100 [00:07<00:42,  1.98it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.sleep.singlesign.00000018.ark 
LOG (copy-feats-to-htk[5.5.1056~1-f6f4c]:main():copy-feats-to-htk.cc:134) 1 HTK feature files generated in the direcory: data/htk
 16%|██████████████████████████████████████▍                                                                                                                                                                                                         | 16/100 [00:08<00:42,  1.98it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.sleep.singlesign.00000001.ark 
LOG (copy-feats-to

...

 45%|████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                                    | 45/100 [00:23<00:27,  1.98it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.telephone.singlesign.00000006.ark 
LOG (copy-feats-to-htk[5.5.1056~1-f6f4c]:main():copy-feats-to-htk.cc:134) 1 HTK feature files generated in the direcory: data/htk
 46%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                 | 46/100 [00:23<00:27,  1.98it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.sleep.singlesign.00000014.ark 
LOG (copy-feat

 60%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                | 60/100 [00:30<00:20,  1.98it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.telephone.singlesign.00000011.ark 
LOG (copy-feats-to-htk[5.5.1056~1-f6f4c]:main():copy-feats-to-htk.cc:134) 1 HTK feature files generated in the direcory: data/htk
 61%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                             | 61/100 [00:31<00:19,  1.98it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.talk.singlesign.00000008.ark 
LOG (copy-feats

 75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                            | 75/100 [00:38<00:12,  1.97it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.telephone.singlesign.00000003.ark 
LOG (copy-feats-to-htk[5.5.1056~1-f6f4c]:main():copy-feats-to-htk.cc:134) 1 HTK feature files generated in the direcory: data/htk
 76%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                         | 76/100 [00:38<00:12,  1.97it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.talk.singlesign.00000015.ark 
LOG (copy-feats

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                        | 90/100 [00:45<00:05,  1.98it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.talk.singlesign.00000002.ark 
LOG (copy-feats-to-htk[5.5.1056~1-f6f4c]:main():copy-feats-to-htk.cc:134) 1 HTK feature files generated in the direcory: data/htk
 91%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                     | 91/100 [00:46<00:04,  1.97it/s]/kaldi/src/featbin/copy-feats-to-htk --output-dir=data/htk --output-ext=htk --sample-period=40000 ark:data/ark/00001.sleep.singlesign.00000009.ark 
LOG (copy-feats-to-

.htk files created
Creating .txt files
Unique Words:  ['puzzle', 'sleep', 'talk', 'telephone', 'tomorrow']
DO
.txt files created


## Training Demo

Run the train command below to train HMM models. The `train_iters` arg determines how many iterations to train to, while `n_splits` determines the number of folds for cross validation. Pass `gmm_mix` as a value that is not `None` to train GMM Mixture Models for emission probabilities. Pass `middle` to the `gmm_pattern` variable to use GMM Mixtures for all but the start/end states. Pass `all` to include GMM Mixtures for all states.

In [9]:
from train import train

train_iters = [150]
leave_one_out = False
n_splits = 20
hmm_step_type = 'single'
gmm_mix = 3
gmm_pattern = 'middle'
random_state = 423

train(
    train_iters=train_iters,
    leave_one_out=leave_one_out, # n_splits would be ignored in this case
    n_splits=n_splits,
    hmm_step_type=hmm_step_type, # see utils/model_utils.py for possible values
    gmm_mix=gmm_mix,
    gmm_pattern=gmm_pattern, # If gmm mix is none this is ignored.
    random_state=random_state
)

You have invoked parallel cross validation. Be prepared for dancing progress bars!
Current split = 18. Current Test data Size = 5. Current Train data Size = 95.
-------------- Training HMM --------------
Running HCompV...
HCompV Complete
Current split = 3. Current Test data Size = 5. Current Train data Size = 95.
-------------- Training HMM --------------
Running HCompV...
HCompV Complete
Current split = 16. Current Test data Size = 5. Current Train data Size = 95.
-------------- Training HMM --------------
Running HCompV...
HCompV Complete
Current split = 1. Current Test data Size = 5. Current Train data Size = 95.
-------------- Training HMM --------------
Running HCompV...
HCompV Complete
Current split = 10. Current Test data Size = 5. Current Train data Size = 95.
-------------- Training HMM --------------
Running HCompV...
HCompV Complete
Current split = 13. Current Test data Size = 5. Current Train data Size = 95.
-------------- Training HMM --------------
Running HCompV...
HComp

 62%|██████▎   | 5/8 [00:04<00:02,  1.13it/s]s]

HRest Complete
Running HERest Iteration: 1...



 88%|████████▊ | 7/8 [00:04<00:00,  1.51it/s]

HRest Complete
Running HERest Iteration: 1...


...

 97%|█████████▋| 144/148 [00:20<00:00,  7.89it/s]

HParse -A -T 1 grammar.txt wordNet.txt 
Creating HParse net from file grammar.txt
Generating Lattice with 10 nodes and 13 links
Writing Word Lattice to wordNet.txt


 99%|█████████▊| 146/148 [00:20<00:00,  7.90it/s]

HParse -A -T 1 grammar.txt wordNet.txt 
Creating HParse net from file grammar.txt
Generating Lattice with 10 nodes and 13 links
Writing Word Lattice to wordNet.txt


100%|██████████| 148/148 [00:20<00:00,  7.20it/s]

## Testing Demo

### Results

The next cell runs the same trained HMMs from the previous step on all of the test data. It outputs a weighted average on results across all folds.

In [10]:
from utils.utils import get_results, clean_and_make_path, get_num_lines
import shutil
import os
import numpy as np
## We can run HVite and HResults on the entire test dataset.

all_results = {'error': [], 'num_rows': []}

for i in range(n_splits):
    fold = str(i)
    
    clean_and_make_path(f'results/{fold}')
    clean_and_make_path(f'hresults/{fold}')
    
    model_file = f'models/{fold}/hmm150/newMacros'
    test_data = f'lists/{fold}/test.data'
    results_file = f'results/{fold}/res_hmm150.mlf'
    hresults_file = f'hresults/{fold}/res_hmm150.txt'
    
    # HVite
    HVite_str = (f'HVite -A -H {model_file} -m -S {test_data} -i '
                    f'{results_file} -p -200 -w wordNet.txt -s 25 dict wordList')

    os.system(HVite_str)

    # HResults
    HResults_str = (f'HResults -A -h -e \\?\\?\\? sil0 -e \\?\\?\\? '
                        f'sil1 -p -t -I all_labels.mlf wordList {results_file} '
                        f'>> {hresults_file}')
    os.system(HResults_str)
    
    results = get_results(hresults_file)
    num_rows = get_num_lines(test_data)
    
    all_results['error'].append(results['error'])
    all_results['num_rows'].append(num_rows)

total = sum(all_results['num_rows'])
print("\nError: ", np.average(all_results['error'], weights=all_results['num_rows']))
    

HVite -A -H models/0/hmm150/newMacros -m -S lists/0/test.data -i results/0/res_hmm150.mlf -p -200 -w wordNet.txt -s 25 dict wordList 
HVite -A -H models/1/hmm150/newMacros -m -S lists/1/test.data -i results/1/res_hmm150.mlf -p -200 -w wordNet.txt -s 25 dict wordList 
HVite -A -H models/2/hmm150/newMacros -m -S lists/2/test.data -i results/2/res_hmm150.mlf -p -200 -w wordNet.txt -s 25 dict wordList 
HVite -A -H models/3/hmm150/newMacros -m -S lists/3/test.data -i results/3/res_hmm150.mlf -p -200 -w wordNet.txt -s 25 dict wordList 
HVite -A -H models/4/hmm150/newMacros -m -S lists/4/test.data -i results/4/res_hmm150.mlf -p -200 -w wordNet.txt -s 25 dict wordList 
HVite -A -H models/5/hmm150/newMacros -m -S lists/5/test.data -i results/5/res_hmm150.mlf -p -200 -w wordNet.txt -s 25 dict wordList 
HVite -A -H models/6/hmm150/newMacros -m -S lists/6/test.data -i results/6/res_hmm150.mlf -p -200 -w wordNet.txt -s 25 dict wordList 
HVite -A -H models/7/hmm150/newMacros -m -S lists/7/test.data 

### Video Samples

The section below contains some video samples from the test data. We play the video with the correct label (from the trained HMMs) on the left side and the incorrect label on the right side. The first cell displays two different words, while the second two cells display the same word, but with a correct and incorrect label, on the left and right respectively.

In [12]:
## Look is displayed on the left and is labeled correctly by the HMM (which is shown below)
## Talk is displayed on the right and is labeled incorrect by the HMM (which is shown below)

from ipywidgets import Output, GridspecLayout
from IPython import display

videos = [
    'split/00001-look-2022_09_21_15_04_44.126-13.mp4',
    'split/00001-talk-2022_09_19_15_42_00.307-5.mp4'
]
grid = GridspecLayout(1, len(videos))

for i, videopath in enumerate(videos):
    out = Output()
    with out:
        display.display(display.Video(videopath, embed=True, height=540, width=360))
    grid[0,i] = out

grid
    

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

HERest Complete
HERest Complete


HERest Complete
HERest Complete


HERest Complete
HERest Complete


HERest Complete
HERest Complete


...

HERest Complete
HERest Complete


HERest Complete


HERest Complete
HERest Complete


HERest Complete


HERest Complete
